# 1. import library and read data

In [1]:
# %load_ext autoreload
# %autoreload 2
import depth_seg_utils
import semantic_seg_utils
from utils import *

import cv2
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import sys
sys.dont_write_bytecode = True

import os
os.getpid()

Using TensorFlow backend.


3168

In [2]:
# read images
import rospy
from cv_bridge import CvBridge
import rosbag
from scipy.spatial.transform import Rotation as R

rosbag_file = "/media/yang/linux2/thesis_test/data/scenenn_231/scenenn_231.bag"
rgb_topic = "/camera/rgb/image_raw"
depth_topic = "/camera/depth/image_raw"
rgb_cam_info_topic = "/camera/rgb/camera_info"
depth_cam_info_topic = "/camera/depth/camera_info"
tf_topic = "/tf"

bridge = CvBridge()
bag = rosbag.Bag(rosbag_file, 'r')

depth_msg_list = []
rgb_msg_list = []

depth_image_list = []
rgb_image_list = []
pose_list = []
tf_list = []
depth_cam_info = None

for topic, msg, t in bag.read_messages(topics = [depth_topic,depth_cam_info_topic,rgb_topic,tf_topic]):
    if topic == depth_cam_info_topic:
        if depth_cam_info is None:
            depth_cam_info = msg
        
    elif topic == depth_topic:
        depth_img = bridge.imgmsg_to_cv2(msg)
        depth_image_list.append(depth_img)
        depth_msg_list.append(msg)

    elif topic == rgb_topic:
        rgb_img = bridge.imgmsg_to_cv2(msg)
        rgb_image_list.append(rgb_img)
        rgb_msg_list.append(msg)

    elif topic == tf_topic:
        tf_list.append(msg)
        translation = msg.transforms[0].transform.translation
        quat = msg.transforms[0].transform.rotation
        r = R.from_quat([quat.x, quat.y, quat.z,quat.w])
        rotation = r.as_dcm()
        T_G_C = np.eye(4).astype(float)
        T_G_C[:3,:3] = rotation
        T_G_C[:3,3] = [translation.x,translation.y,translation.z]
        pose_list.append(T_G_C)

# 2. Semantic and depth segmentation

In [3]:

# initialized segmentors
height = depth_image_list[0].shape[0]
width = depth_image_list[0].shape[1]
K_depth = np.array(depth_cam_info.K).reshape(3,3)
segmentor = depth_seg_utils.depth_segmentation_py.DepthSegmentation_py(height,width,cv2.CV_32FC1, K_depth)

node = semantic_seg_utils.MaskRCNNNode()


Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERENC

# 3. map integration

In [4]:
test = frame2Segments(segmentor, node, depth_image_list[0], rgb_image_list[0],pose_list[0] )

In [7]:
# inspect segments
seg_test = test[0]
point_distance = np.linalg.norm(seg_test.points, axis=1)
print("max dis: ", np.max(point_distance))
print("mean dis: ", np.mean(point_distance))
print("median dis: ", np.median(point_distance))

('max dis: ', 2.1940608072590515)
('mean dis: ', 1.8363727359320507)
('median dis: ', 1.8185875611457971)


In [5]:
import sys
import cv2
import numpy as np
import os
# import python module of depth_segmentation
lib_path = "/home/yang/thesis_test/toolbox/voxbloxpp_ws/devel/lib"
sys.path.append(lib_path)

import gsm_py


In [6]:
log_file = os.path.abspath("./result")
gsm_node = gsm_py.GlobalSegmentMap_py(log_file)

In [7]:
log_file

'/home/yang/thesis_test/toolbox/voxbloxpp_ws/src/voxblox-plusplus/global_segment_map_py/scripts/result'

In [8]:
for segment in test[1:]:
    gsm_node.insertSegments(segment.points,segment.colors, segment.geometry_confidence,
        segment.instance_label,segment.class_label, segment.label_confidence,segment.pose)

In [8]:
print(segment.points.shape)
print(segment.colors.shape)
print(segment.geometry_confidence.shape)
print(segment.instance_label)
print(segment.label_confidence)
print(segment.pose.dtype)

(26622, 3)
(26622, 3)
(26622, 1)
0
1
float32


In [ ]:
gsm_node.generateMesh(log_file)

: 

: 